#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Sep  4 10:23:50 2018
Modified Oct 2020, Oct 2021, Oct 2023

@author: Said Ladjal,Isabelle Bloch
"""

In [ ]:
#%% SECTION 1 -- inclusion of packages


import numpy as np
import platform
import tempfile
import os
import matplotlib.pyplot as plt
from scipy import ndimage as ndip
from skimage import io as skio


import skimage.morphology as morpho 
from skimage.segmentation import watershed 
from skimage.draw import line
import skimage.feature as skf
from scipy import ndimage as ndi

In [ ]:
#%% SECTION 2 -- Useful functions

def viewimage(im,normalise=True,MINI=0.0, MAXI=255.0):
    """ Cette fonction fait afficher l'image EN NIVEAUX DE GRIS 
        dans gimp. Si un gimp est deja ouvert il est utilise.
        Par defaut normalise=True. Et dans ce cas l'image est normalisee 
        entre 0 et 255 avant d'être sauvegardee.
        Si normalise=False MINI et MAXI seront mis a 0 et 255 dans l'image resultat

        Questa funzione mostra l'immagine IN SCALA DI GRIGI in GIMP. Se GIMP 
        è già aperto, verrà utilizzato. Per impostazione predefinita, normalize=True.
        In questo caso, l'immagine viene normalizzata tra 0 e 255 prima di essere salvata. Se normalize=False, i valori MIN e MAX saranno impostati su 0 e 255 nell'immagine risultante.
        
    """
    imt=np.float32(im.copy())
    if platform.system()=='Darwin': #on est sous mac
        prephrase='open -a GIMP '
        endphrase=' ' 
    elif platform.system()=='Linux': #SINON ON SUPPOSE LINUX (si vous avez un windows je ne sais comment faire. Si vous savez dites-moi.)
        prephrase='gimp '
        endphrase= ' &'
    elif platform.system()=='Windows':
        prephrase='start /B "D:/GIMP/bin/gimp-2.10.exe" -a '#Remplacer D:/... par le chemin de votre GIMP
        endphrase= ''
    else:
        print('Systeme non pris en charge par l affichage GIMP')
        return 'erreur d afficahge'
    if normalise:
        m=imt.min()
        imt=imt-m
        M=imt.max()
        if M>0:
            imt=imt/M

    else:
        imt=(imt-MINI)/(MAXI-MINI)
        imt[imt<0]=0
        imt[imt>1]=1
    
    nomfichier=tempfile.mktemp('TPIMA.png')
    commande=prephrase +nomfichier+endphrase
    skio.imsave(nomfichier,imt)
    os.system(commande)


def viewimage_color(im,normalise=True,MINI=0.0, MAXI=255.0):
    """ Cette fonction fait afficher l'image EN NIVEAUX DE GRIS 
        dans gimp. Si un gimp est deja ouvert il est utilise.
        Par defaut normalise=True. Et dans ce cas l'image est normalisee 
        entre 0 et 255 avant d'être sauvegardee.
        Si normalise=False MINI(defaut 0) et MAXI (defaut 255) seront mis a 0 et 255 dans l'image resultat
        

        Questa funzione mostra l'immagine IN SCALA DI GRIGI in GIMP. 
        Se GIMP è già aperto, verrà utilizzato. Per impostazione predefinita, 
        normalise=True. In questo caso, l'immagine viene normalizzata tra 0 
        e 255 prima di essere salvata. Se normalise=False, i valori MINI 
        (impostati per default a 0) e MAXI (impostati per default a 255) 
        saranno impostati a 0 e 255 nell'immagine risultante.
    """
    imt=np.float32(im.copy())
    if platform.system()=='Darwin': #on est sous mac
        prephrase='open -a GIMP '
        endphrase=' ' 
    elif platform.system()=='Linux': #SINON ON SUPPOSE LINUX (si vous avez un windows je ne sais comment faire. Si vous savez dites-moi.)
        prephrase='gimp '
        endphrase= ' &'
    elif platform.system()=='Windows':
        prephrase='start /B "D:/GIMP/bin/gimp-2.10.exe" -a '#Remplacer D:/... par le chemin de votre GIMP
        endphrase= ''
    else:
        print('Systeme non pris en charge par l affichage GIMP')
        return 'erreur d afficahge'
    
    if normalise:
        m=imt.min()
        imt=imt-m
        M=imt.max()
        if M>0:
            imt=imt/M
    else:
        imt=(imt-MINI)/(MAXI-MINI)
        imt[imt<0]=0
        imt[imt>1]=1
    
    nomfichier=tempfile.mktemp('TPIMA.pgm')
    commande=prephrase +nomfichier+endphrase
    skio.imsave(nomfichier,imt)
    os.system(commande)


def strel(forme,taille,angle=45):
    """renvoie un element structurant de forme  
     'diamond'  boule de la norme 1 fermee de rayon taille
     'disk'     boule de la norme 2 fermee de rayon taille
     'square'   carre de cote taille (il vaut mieux utiliser taille=impair)
     'line'     segment de langueur taille et d'orientation angle (entre 0 et 180 en degres)
      (Cette fonction n'est pas standard dans python)

      Restituisce un elemento strutturante di forma:

'diamond': una sfera chiusa di norma 1 con un raggio di dimensione
'disk': una sfera chiusa di norma 2 con un raggio di dimensione
'square': un quadrato di lato di dimensione (è meglio usare dimensione dispari)
'line': un segmento di lunghezza dimensione e orientamento angolo 
        (compreso tra 0 e 180 gradi)
    """

    if forme == 'diamond':
        return morpho.diamond(taille)
    if forme == 'disk':
        return morpho.disk(taille)
    if forme == 'square':
        return morpho.square(taille)
    if forme == 'line':
        angle=int(-np.round(angle))
        angle=angle%180
        angle=np.float32(angle)/180.0*np.pi
        x=int(np.round(np.cos(angle)*taille))
        y=int(np.round(np.sin(angle)*taille))
        if x**2+y**2 == 0:
            if abs(np.cos(angle))>abs(np.sin(angle)):
                x=int(np.sign(np.cos(angle)))
                y=0
            else:
                y=int(np.sign(np.sin(angle)))
                x=0
        rr,cc=line(0,0,y,x)
        rr=rr-rr.min()
        cc=cc-cc.min()
        img=np.zeros((rr.max()+1,cc.max()+1) )
        img[rr,cc]=1
        return img
    raise RuntimeError('Erreur dans fonction strel: forme incomprise')

            

def couleurs_alea(im):
    """ 
    Donne des couleurs aleatoires a une image en niveau de gris.
    Cette fonction est utile lorsque le niveua de gris d'interprete comme un numero
      de region. Ou encore pour voir les leger degrades d'une teinte de gris.

    Assegna colori casuali a un'immagine in scala di grigi. Questa funzione 
    è utile quando il livello di grigio è interpretato come un numero di regione 
    o per visualizzare leggeri sfumature di una tonalità di grigio.
      """
    sh=im.shape
    out=np.zeros((sh[0],sh[1],3),dtype=np.uint8)
    nbcoul=np.int32(im.max())
    tabcoul=np.random.randint(0,256,size=(nbcoul+1,3))
    tabcoul[0,:]=0
    for k in range(sh[0]):
        for l in range(sh[1]):
            out[k,l,:]=tabcoul[im[k,l]]
    return out

def gris_depuis_couleur(im):
    """ Transforme une image couleur en image a niveaux de gris"""
    return im[:,:,:3].sum(axis=2)/3

In [ ]:
#%% SECTION 3 -- Examples of functions for this work

# Binary images 
#im1=skio.imread('images/cellbin.bmp')
im2=skio.imread('images/cafe.bmp')*255


# Gray-scale images
#im=skio.imread('images/retina2.gif')
#im=skio.imread('bat200.bmp')
#im=skio.imread('bulles.bmp')
#im=gris_depuis_couleur (skio.imread('cailloux.png'))
#im=gris_depuis_couleur(skio.imread('cailloux2.png'))
#im=skio.imread('laiton.bmp')
#print(im.max())

plt.imshow(im,cmap="gray",vmin=0, vmax=255)

# viewimage(im) - Utilisable à la place de plt.imshow si Gimp est installé.

se=strel('disk',4)

# dilation
dil=morpho.dilation(im,se)
plt.imshow(dil,cmap="gray",vmin=0, vmax=255)
plt.savefig('grafico.png')#mia aggiunta

# erosion
ero=morpho.erosion(im,se)
plt.imshow(ero,cmap="gray",vmin=0, vmax=255)

#opening
open=morpho.opening(im,se)
plt.imshow(open,cmap="gray",vmin=0, vmax=255)

#closing
close=morpho.closing(im,se)
plt.imshow(close,cmap="gray",vmin=0, vmax=255)

In [ ]:
#%% Top-hat transforme
im=skio.imread('images/retina2.gif')
t=10
se=strel('line',t,-45)
ch=im-morpho.opening(im,se)
plt.imshow(ch,cmap="gray",vmin=0, vmax=255)

#%%  Alternate sequential filter
im=skio.imread('images/retina2.gif')
imt=im.copy()
N=3
for k in range(N):
    se=strel('disk',k)
    imt=morpho.closing(morpho.opening(imt,se),se)
plt.imshow(imt,cmap="gray",vmin=0, vmax=255)

In [ ]:
#%% Watersheds
im=skio.imread('bat200.bmp')
se=morpho.disk(1)

grad=morpho.dilation(im,se)-morpho.erosion(im,se)
grad=np.int32(grad>40)*grad
plt.imshow(grad,cmap="gray")

local_mini = skf.peak_local_max(255-grad, #il n'y a pas de fonction local_min...
                            indices=False)
markers = ndi.label(local_mini)[0]
plt.imshow(local_mini,cmap="gray")

labels = watershed(grad, markers,watershed_line=True)
plt.imshow(couleurs_alea(labels))
# viewimage_color(couleurs_alea(labels)) - Utilisable si gimp est installé

# visualization of the result
segm=labels.copy()
for i in range(segm.shape[0]):
    for j in range(segm.shape[1]):
        if segm[i,j] == 0: 
            segm[i,j]=255
        else:
            segm[i,j]=0
#superimposition of segmentation contours on the original image
contourSup=np.maximum(segm,im)
plt.imshow(contourSup,cmap="gray") 

In [ ]:
#%% reconstruction
im=skio.imread('retina2.gif')
se4=strel('disk',4)
open4=morpho.opening(im,se4)
reco=morpho.reconstruction(open4,im)
plt.imshow(reco,cmap="gray")


In [ ]:
#%% TP1 MORPHO

#dilation with different structuring elements in terms of size and shape
for se_str in ["diamond", "disk", "square", "line"]:
    size=4
    se=strel(se_str,size)
    im_cell_bin=skio.imread('images/cellbin.bmp')*255
    dil=morpho.dilation(im_cell_bin,se)
    plt.title("original image of a cell bin")
    plt.imshow(im_cell_bin,cmap="gray",vmin=0, vmax=255)
    plt.show()
    plt.title(f"image of a cell bin after dilation with a {se_str} of size {size}")
    plt.imshow(dil,cmap="gray",vmin=0, vmax=255)